In [ ]:
import torchvision

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data import *
from local.notebook.showdoc import show_doc

In [ ]:
import string
ds = list(string.ascii_lowercase)

In [ ]:
dl = DataLoader(ds, batch_size=8)
' '.join(''.join(o) for o in dl)

'abcdefgh ijklmnop qrstuvwx yz'

In [ ]:
dl = DataLoader(ds, batch_size=8, drop_last=True)
' '.join(''.join(o) for o in dl)

'abcdefgh ijklmnop qrstuvwx'

In [ ]:
dl = DataLoader(ds, batch_size=8, shuffle=True)
' '.join(''.join(o) for o in dl)

'jmfunarh sldwkypo qictexvg bz'

In [ ]:
dl = DataLoader(ds, batch_size=8, sampler=SequentialSampler(ds))
' '.join(''.join(o) for o in dl)

'abcdefgh ijklmnop qrstuvwx yz'

In [ ]:
dl = DataLoader(ds, batch_sampler=BatchSampler(SequentialSampler(ds), 8, False))
' '.join(''.join(o) for o in dl)

'abcdefgh ijklmnop qrstuvwx yz'

In [ ]:
def rev_collate(s): return default_collate(list(reversed(s)))

dl = DataLoader(ds, batch_size=8, collate_fn=rev_collate)
' '.join(''.join(o) for o in dl)

'hgfedcba ponmlkji xwvutsrq zy'